# Project: Horse or Human Classification using CNN

### Spliting dataset for training:
### 500 horse images
### 527 male and female human images

### Spliting dataset for Testing & Validation:
### 122 horse images
## 123 male and female human images

## Problem Statement:
### I need to classify whether the given image is human or horse.

### Solution:
### In order to solve this problem, I am going to use CNN (Convolutional Neural Network) of Deep Learning. 


## Importing required Libraries



In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import os

## Load Data

In [ ]:
train_folder = "horse-or-human/dataset/train"
validation_folder = "horse-or-human/dataset/validation"

train_data_path = os.path.join(train_folder)
validation_data_path = os.path.join(validation_folder)

## Data Preprocessing

In [ ]:
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

In [ ]:
training_data = training_datagen.flow_from_directory(
    train_folder,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_data =  valid_datagen.flow_from_directory(
    validation_folder, 
    target_size=(150, 150), 
    batch_size=32,
    class_mode='binary')

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

images = [training_data[0][0][0] for i in range(5)]
plotImages(images)


# Building cnn model

In [ ]:
cnn_model = keras.models.Sequential([
                                    keras.layers.Conv2D(filters=32, kernel_size=7, input_shape=[150, 150, 3],kernel_regularizer=l2(l=0.01)),
                                    
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                     
                                    keras.layers.Conv2D(filters=64, kernel_size=5),
                                    
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                     
                                    keras.layers.Conv2D(filters=128, kernel_size=3),
                                    
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                                                         
                                    keras.layers.Conv2D(filters=256, kernel_size=3),
                                    
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                                                     
                                    keras.layers.Flatten(), # neural network beulding
                                    keras.layers.Dense(units=128, activation='relu'), # input layers
                                    
                                    keras.layers.Dropout(0.5),                                      
                                    keras.layers.Dense(units=256, activation='relu'),  
                                                                      
                                    keras.layers.Dropout(0.5),                                    
                                    keras.layers.Dense(units=2, activation='softmax') # output layer
])

In [ ]:
cnn_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, input_shape=[150, 150, 3]),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=3),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=3),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(), # neural network beulding
    keras.layers.Dense(units=128, activation='relu'), # input layers
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='sigmoid') # output layer
])


# Compiling the model

In [ ]:
cnn_model.compile(optimizer = Adam(lr=0.0001), 
                  loss = 'binary_crossentropy', 
                  metrics = ['accuracy'])

# Training the model

In [ ]:
history = cnn_model.fit(
    training_data,
    steps_per_epoch = len(training_data),
    epochs = 30,
    validation_data = valid_data,
    validation_steps = len(valid_data))

In [ ]:
history.history

# summarize history for accuracy

In [ ]:
print(history)
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot([history.history['loss'],history.history['accuracy']])
plt.title('model accuracy vs loss')
plt.ylabel('loss vs accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Plotting the accuracy and loss charts

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(loc=0)
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend(loc=0)
plt.figure()

test_folder = "horse-or-human/dataset/validation"
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
test_folder,
target_size=(150, 150),
batch_size=32,
class_mode='binary')

# Evaluating the model

In [ ]:
scores = cnn_model.evaluate(test_data, verbose=1)
print('Test accuracy:', scores[1])

# Saving the model

In [ ]:
cnn_model.save("horse_or_human_model.h5")